In [1]:
from tms_risk.utils.data import get_all_behavior, get_subjects, get_participant_info
import seaborn as sns
import pingouin
import matplotlib.pyplot as plt
import os.path as op
import pandas as pd
import numpy as np

bids_folder = '/data/ds-tmsrisk'

In [2]:
df_wo_outliers =  get_all_behavior(bids_folder='/data/ds-tmsrisk', all_tms_conditions=True, exclude_outliers=True)
df_w_outliers =  get_all_behavior(bids_folder='/data/ds-tmsrisk', all_tms_conditions=True, exclude_outliers=False)

In [3]:
df_w_outliers.loc[[22, 49]].groupby(['risky_first', 'stimulation_condition'])[['chose_risky']].mean()

chose_risky
risky_first stimulation_condition             
False       baseline                  0.233333
            ips                       0.088496
            vertex                    0.093220
True        baseline                  0.200000
            ips                       0.075630
            vertex                    0.067227

In [4]:
df = pd.concat((df_wo_outliers, df_w_outliers), keys=['Without outliers', 'with outliers'], names=['outliers'], axis=0).drop('baseline', level='stimulation_condition')

In [5]:
df.groupby(['outliers', 'subject']).size().groupby('outliers').size()

outliers
Without outliers    35
with outliers       37
dtype: int64

In [6]:
df.groupby(['subject'])['chose_risky'].mean().sort_values()

subject
22    0.066946
49    0.095652
59    0.242678
74    0.270833
9     0.354167
72    0.359833
6     0.383333
29    0.394958
35    0.397490
37    0.437500
4     0.448718
21    0.466667
19    0.468619
67    0.518828
63    0.558442
34    0.562500
26    0.570833
1     0.577406
46    0.581590
36    0.587500
11    0.589958
18    0.591489
2     0.612069
47    0.614407
50    0.616667
3     0.616667
53    0.619247
25    0.647059
69    0.654167
56    0.673640
31    0.682008
7     0.682203
45    0.706383
10    0.733333
30    0.744770
5     0.783333
62    0.857759
Name: chose_risky, dtype: float64

In [7]:
import pingouin

In [8]:
from IPython.display import display

for outliers, d in df.groupby(['outliers']):

    print(outliers)
    print(d.groupby(['subject', 'stimulation_condition'])[['chose_risky']].mean().groupby(['stimulation_condition']).agg(['mean', 'std']) * 100)

    tmp = d.groupby(['subject', 'stimulation_condition'])[['chose_risky', 'rt']].mean().reset_index()



    display(pingouin.rm_anova(data=tmp.reset_index(), dv='chose_risky', within='stimulation_condition', subject='subject'))
    display(pingouin.rm_anova(data=tmp.reset_index(), dv='rt', within='stimulation_condition', subject='subject'))

    tmp = d.groupby(['subject', 'stimulation_condition', 'risky_first'])[['chose_risky', 'rt']].mean().reset_index()
    display(pingouin.rm_anova(data=tmp.reset_index(), dv='chose_risky', within=['stimulation_condition', 'risky_first'], subject='subject'))
    display(pingouin.rm_anova(data=tmp.reset_index(), dv='rt', within=['stimulation_condition', 'risky_first'], subject='subject'))


    tmp = d.pivot_table(index=['subject', 'risky_first'], columns='stimulation_condition', values='chose_risky')
    display(tmp.groupby(['risky_first']).mean())
    # tmp1 = tmp.xs(False, 0, 'risky_first')
    # tmp2 = tmp.xs(True, 0, 'risky_first')

    # pingouin.ttest(tmp1['ips'], tmp1['vertex'], paired=True)

    display(tmp.groupby('risky_first').apply(lambda x: pingouin.ttest(x['ips'], x['vertex'], paired=True)))

('Without outliers',)
                      chose_risky           
                             mean        std
stimulation_condition                       
ips                     57.484034  15.099787
vertex                  54.565327  16.327991


,Source,ddof1,ddof2,F,p-unc,ng2,eps
0,stimulation_condition,1,34,1.792451,0.189511,0.008787,1.0


,Source,ddof1,ddof2,F,p-unc,ng2,eps
0,stimulation_condition,1,34,0.349162,0.558496,0.000817,1.0


/Users/gdehol/mambaforge/envs/tms_risk/lib/python3.11/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
/Users/gdehol/mambaforge/envs/tms_risk/lib/python3.11/site-packages/pingouin/distribution.py:508: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  .diff(axis=1)


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,stimulation_condition,0.029779,1,34,0.029779,1.792365,0.189522,0.189522,0.007578,1.0
1,risky_first,0.120497,1,34,0.120497,10.224865,0.002993,0.002993,0.029971,1.0
2,stimulation_condition * risky_first,0.019162,1,34,0.019162,4.999731,0.032022,0.032022,0.004889,1.0


...

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,stimulation_condition,0.028361,1,36,0.028361,1.801340,0.187954,0.187954,0.004999,1.0
1,risky_first,0.118335,1,36,0.118335,10.544190,0.002524,0.002524,0.020534,1.0
2,stimulation_condition * risky_first,0.017559,1,36,0.017559,4.782477,0.035331,0.035331,0.003101,1.0


/Users/gdehol/mambaforge/envs/tms_risk/lib/python3.11/site-packages/pingouin/distribution.py:507: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  data.groupby(level=1, axis=1, observed=True, group_keys=False)
/Users/gdehol/mambaforge/envs/tms_risk/lib/python3.11/site-packages/pingouin/distribution.py:508: FutureWarning: DataFrameGroupBy.diff with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  .diff(axis=1)


,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
0,stimulation_condition,0.010990,1,36,0.010990,0.374009,0.544670,0.544670,0.000754,1.0
1,risky_first,0.038311,1,36,0.038311,5.116079,0.029841,0.029841,0.002623,1.0
2,stimulation_condition * risky_first,0.002158,1,36,0.002158,0.691908,0.411000,0.411000,0.000148,1.0


stimulation_condition,ips,vertex
risky_first,,
False,0.587372,0.537902
True,0.509034,0.503133


,,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
risky_first,,,,,,,,,
False,T-test,2.271174,36,two-sided,0.029216,"[0.01, 0.09]",0.258805,1.706,0.334696
True,T-test,0.246088,36,two-sided,0.807012,"[-0.04, 0.05]",0.028844,0.182,0.053349


In [9]:
pingouin.pairwise_tests(data=tmp.reset_index(), dv='chose_risky', within=['stimulation_condition', 'risky_first'], subject='subject', padjust='fdr_bh')

KeyError: 'chose_risky'

In [ ]:
tmp

,subject,stimulation_condition,risky_first,chose_risky,rt
0,1,ips,False,0.666667,1.474453
1,1,ips,True,0.583333,1.367737
2,1,vertex,False,0.559322,1.322484
3,1,vertex,True,0.500000,1.308091
4,2,ips,False,0.750000,0.459204
...,...,...,...,...,...
143,72,vertex,True,0.186441,0.843889
144,74,ips,False,0.216667,0.677283
145,74,ips,True,0.350000,0.605492
146,74,vertex,False,0.216667,0.665002


In [ ]:
tmp = d.groupby(['subject', 'risky_first', 'stimulation_condition'])[['chose_risky']].mean()

In [ ]:
tmp.unstack(['stimulation_condition'])

chose_risky          
stimulation_condition         ips    vertex
subject risky_first                        
1       False            0.666667  0.559322
        True             0.583333  0.500000
2       False            0.750000  0.568966
        True             0.500000  0.629630
3       False            0.633333  0.516667
...                           ...       ...
69      True             0.683333  0.716667
72      False            0.466667  0.366667
        True             0.416667  0.186441
74      False            0.216667  0.216667
        True             0.350000  0.300000

[74 rows x 2 columns]

In [10]:
tmp.unstack(['stimulation_condition']).droplevel(0, 1).groupby(['risky_first']).apply(lambda d: pingouin.ttest(d['ips'], d['vertex'], paired=True))

KeyError: 'Level stimulation_condition not found'

In [11]:
pingouin.pairwise_ttests(data=tmp, dv='chose_risky', within=['stimulation_condition', 'risky_first'], subject='subject')

/Users/gdehol/mambaforge/envs/tms_risk/lib/python3.11/site-packages/pingouin/pairwise.py:28: UserWarning: pairwise_ttests is deprecated, use pairwise_tests instead.
  warnings.warn("pairwise_ttests is deprecated, use pairwise_tests instead.", UserWarning)


KeyError: 'chose_risky'

In [ ]:
d_with